In [24]:
#https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/

import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt


pd.options.display.max_seq_items = 2000
pd.options.display.max_colwidth = 90
pd.options.display.max_rows = 999

#https://altair-viz.github.io/gallery/errorbars_with_std.html
#import altair as alt
#from vega_datasets import data

issues = '../data/issue_all_jabref.csv'
dataIssues = pd.read_csv(issues,sep=',')

pull_request = '../data/pr_all_jabref.csv'
dataPR = pd.read_csv(pull_request,sep=',')
dataPRIssues = pd.read_csv(pull_request,sep=',')

commits = '../data/commits_all_jabref.csv'
dataCommit = pd.read_csv(commits,sep=',', lineterminator='\n') #linetermination needed if file contains '\r' in places

In [48]:
#next cell will change this data
df = dataCommit
print(df.shape)
print(df.columns)
df.head(500)

(8431, 11)
Index(['issue_number', 'sha', 'message', 'author_name', 'date', 'additions',
       'deletions', 'changes', 'file_list', 'status', 'patch_text\r'],
      dtype='object')


,issue_number,sha,message,author_name,date,additions,deletions,changes,file_list,status,patch_text\r
19,1,6db780e96882aca9fa3170afd13a7965dfb61a69,Merge branch 'sorting',Olaf Lenz,2014-03-12T09:16:45Z,1157,749,1906,src/java/net/sf/jabref/FileSortTab.java;src/java/net/sf/jabref/FileTab.java;src/java/n...,added;modified;modified;modified;modified;modified;modified;modified,"@@ -0,0 +1,387 @@\n+/* Copyright (C) 2013 JabRef contributors.\n+ This program is fre..."
21,2,a66468793a5dcba41f0f9307f8d762baecc7f331,Gradle build works basically. Reuses existing folder structure.,Simon Harrer,2014-03-12T17:20:08Z,72,2,74,build-wrapper.xml;build.gradle;src/extensions/SimpleCsvImporter.java,added;added;modified,"@@ -0,0 +1,3 @@\n+<project default=""test"">\n+ <include file=""build.xml"" as=""antTarg..."
22,3,0783fd8ea77776e69372e60bde2df3d31abf852e,Some example new Dutch translations via the GitHub web page.,Egon Willighagen,2014-03-15T09:33:10Z,13,13,26,src/resource/JabRef_nl.properties,modified,"@@ -34,7 +34,7 @@ Abbreviate_journal_names_of_the_selected_entries_(ISO_abbreviation)=..."
23,4,71357a4f5234b5eed78eb6eba9d50a0563398cb4,Spanish translation update,jtornero,2014-03-17T21:28:26Z,3335,3335,6670,src/resource/JabRef_da.properties;src/resource/JabRef_de.properties;src/resource/JabRe...,modified;modified;modified;modified;modified;modified;modified;modified;modified;modif...,"@@ -1,5 +1,5 @@\n #!\n-#! created/edited by Popeye version 0.54 (popeye.sourceforge.ne..."
24,5,5eb77ba20c79e6e53e0ab04c2da502288928a87a,Update JabRef_in.properties\n\nIndonesian translation added,was123,2014-03-18T00:01:58Z,4,4,8,src/resource/JabRef_in.properties,modified,"@@ -1304,10 +1304,10 @@ Style_selection=Pilihan_gaya\n The_panel_below_shows_the_defin..."
25,6,57427502055b834b2cbad35f3514ec8003c5ac7a,Fixed splash-dev.svg,Olaf Lenz,2014-03-19T19:41:29Z,30,29,59,src/images/splash-dev.svg,modified,"@@ -79,18 +79,18 @@\n width=""572.00000""\n patternUnits=""userSpaceOnUse"">..."
26,7,233203728c3168172527970b53b311be27f75474,I have implemented 2 features requested in Ticket #841.\n\n The user can now speci...,noravanq,2014-03-23T04:48:49Z,213,6,219,src/main/java/net/sf/jabref/GUIGlobals.java;src/main/java/net/sf/jabref/JabRefPreferen...,modified;modified;modified;modified;modified;modified,"@@ -28,6 +28,8 @@\n import javax.swing.ImageIcon;\n import javax.swing.JLabel;\n \n+im..."
28,8,2360cb74d2e8d88e1bc705db36e885e44607642f,Erase fields listed in XMP Privacy Settings from PDF DocumentInformation when XMP-tagging,Adrian Daerr,2014-04-22T18:19:15Z,14,1,15,src/main/java/net/sf/jabref/util/XMPUtil.java,modified,"@@ -996,7 +996,20 @@ public static void writeDocumentInformation(PDDocument document,\..."
30,9,57cbe70e1be14f7b8f8754e05f6b722ed72ce887,add CSIRO copyright 2014,Peter Ansell,2014-05-07T01:02:09Z,1,1,2,src/main/java/net/sf/jabref/external/ACSPdfDownload.java,modified,"@@ -1,4 +1,4 @@\n-/* Copyright (C) 2003-2011 JabRef contributors.\n+/* Copyright (C)..."
34,10,b6a89a81a983332d35420534a766d8dfabb6b81e,fix bug causing double entries.,Ling Wang,2014-05-20T02:56:41Z,1099,1,1100,src/main/java/net/sf/jabref/BibtexEntry.java;src/main/java/net/sf/jabref/BibtexEntryTy...,modified;modified;added;added;added,"@@ -421,8 +421,10 @@ public void write(Writer out, FieldFormatter ff, boolean write) t..."


In [47]:
#drop any multiple commits for a pr, keeping only the last one. 
#assumes the commits are listed in order by date.
df.drop_duplicates(subset='issue_number',keep='last',inplace=True)

In [5]:
#converting Float to Int 
dataIssues['issue_number'] = dataIssues['issue_number'].fillna(0).astype(np.int64)

#renaming columns and excluding unecessary data

#commits needs only pr/issue number, commit author, commit date, and commit message.
dataCommit.columns = ['PR_Number','sha(not needed)', 'Commit_Message', 'Commit_Author', 'Commit_Date', 'additions(not needed)', 'deletions(not needed)', 'changes(not needed)', 'file_list(not needed)', 'status(not needed)', 'patch_text\r(not needed)']
dataCommit.drop(['sha(not needed)', 'additions(not needed)', 'deletions(not needed)', 'changes(not needed)', 'file_list(not needed)', 'status(not needed)', 'patch_text\r(not needed)'], axis=1, inplace=True)

#prs needs only pr/issue number, pr closed date, pr author, pr title, pr body, and pr comments.
#This data is missing pr author so author login will represent it instead
dataPR.columns = ['PR_Number', 'PR_Title', 'pr_author(empty)', 'PR_Author', 'PR_Closed_Date', 'PR_Body', 'PR_Comments']
del dataPR['pr_author(empty)']

#issues needs only pr/issue number, issue closed date, issue author, issue title, issue body, and issue comments
#issue author is also empty here
dataIssues.columns = ['PR_Number', 'Issue_Title', 'issue_author(empty)', 'Issue_Author', 'Issue_Closed_Date', 'Issue_Body', 'Issue_Comments']
del dataIssues['issue_author(empty)']

##only necesary if issues data does not already contain PRs
dataPRIssues.columns = ['PR_Number', 'Issue_Title', 'issue_author(empty)', 'Issue_Author', 'Issue_Closed_Date', 'Issue_Body', 'Issue_Comments']
del dataPRIssues['issue_author(empty)']

In [6]:
df = dataPR
print(df.shape)
print(df.columns)
df.head(500)

(8446, 6)
Index(['PR_Number', 'PR_Title', 'PR_Author', 'PR_Closed_Date', 'PR_Body',
       'PR_Comments'],
      dtype='object')


,PR_Number,PR_Title,PR_Author,PR_Closed_Date,PR_Body,PR_Comments
0,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN
1,2,Basic gradle integration,simonharrer,2014-03-12T17:35:16Z,This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN
2,3,Some example new Dutch translations via the GitHub web page.,egonw,2014-03-15T09:33:22Z,NaN,NaN
3,4,Spanish translation update,jtornero,2014-03-17T21:29:24Z,Three new strings translated.\n,"Even though the other translation files have been updated, I accept this pull request...."
4,5,Update JabRef_in.properties,was123,2014-03-18T00:02:27Z,Indonesian translation added\n,NaN
5,6,Fixed splash-dev.svg,olenz,2014-03-19T19:44:35Z,NaN,NaN
6,7,I have implemented 2 features requested in Ticket #841.,noravanq,2014-03-23T04:53:18Z,"```\nThe user can now specify an arbitrary number of file-link columns in te ""Entry ta...",NaN
7,8,PDF-file metadata: Privacy Filtering all metadata,adaerr,2014-04-23T12:58:24Z,This pull-request pertains to the addition of metadata to PDF files associated with en...,thx!\n
8,9,Support FindFullText with ACS DOIs,ansell,2014-05-06T06:36:03Z,Adds a FullTextFinder implementation to transform the ACS DOI redirect URLs to their P...,NaN
9,10,try to fix some obvious bugs about `groups`,braindevices,2014-05-08T03:14:52Z,"Hi All,\nI just upgraded to 2.10 and I found some bugs immediately.\n1. in the right-c...",After 2.10 the BibtexEntryType.java is changed a lot. The optional fields and required...


In [7]:
#merging dataframes
pull_commit = pd.merge(dataPR, dataCommit, on=['PR_Number','PR_Number'])
pull_issue = pd.concat([dataPRIssues, dataIssues], axis=0)
pull_issue.sort_values(by='PR_Number', inplace=True)

In [8]:
pull_commit.to_csv('pull_commit.csv', sep=',', encoding='utf-8', index=False)

pull_issue.to_csv('issues.csv', sep=',', encoding='utf-8', index=False)


In [9]:
data_Issues = './issues.csv'
dataIssues = pd.read_csv(data_Issues, sep=',' )

data_PullCommit = './pull_commit.csv'
dataPullCommit = pd.read_csv(data_PullCommit, sep=',' )

In [10]:
dataIssues.head(100)

,PR_Number,Issue_Title,Issue_Author,Issue_Closed_Date,Issue_Body,Issue_Comments
0,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN
1,2,Basic gradle integration,simonharrer,2014-03-12T17:35:16Z,This adds basic gradle integration. The project files for Intellij and Eclipse can be ...,NaN
2,3,Some example new Dutch translations via the GitHub web page.,egonw,2014-03-15T09:33:22Z,NaN,NaN
3,4,Spanish translation update,jtornero,2014-03-17T21:29:24Z,Three new strings translated.\n,"Even though the other translation files have been updated, I accept this pull request...."
4,5,Update JabRef_in.properties,was123,2014-03-18T00:02:27Z,Indonesian translation added\n,NaN
5,6,Fixed splash-dev.svg,olenz,2014-03-19T19:44:35Z,NaN,NaN
6,7,I have implemented 2 features requested in Ticket #841.,noravanq,2014-03-23T04:53:18Z,"```\nThe user can now specify an arbitrary number of file-link columns in te ""Entry ta...",NaN
7,8,PDF-file metadata: Privacy Filtering all metadata,adaerr,2014-04-23T12:58:24Z,This pull-request pertains to the addition of metadata to PDF files associated with en...,thx!\n
8,9,Support FindFullText with ACS DOIs,ansell,2014-05-06T06:36:03Z,Adds a FullTextFinder implementation to transform the ACS DOI redirect URLs to their P...,NaN
9,10,try to fix some obvious bugs about `groups`,braindevices,2014-05-08T03:14:52Z,"Hi All,\nI just upgraded to 2.10 and I found some bugs immediately.\n1. in the right-c...",After 2.10 the BibtexEntryType.java is changed a lot. The optional fields and required...


In [11]:
dataPullCommit.head(500)

,PR_Number,PR_Title,PR_Author,PR_Closed_Date,PR_Body,PR_Comments,Commit_Message,Commit_Author,Commit_Date
0,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,"Adding private fields to SQL scripts in order to export/import owner, timestamp, etc.",Igor Steinmacher,2013-10-23T19:13:12Z
1,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New entries translated to pt-BR,Igor Steinmacher,2013-10-23T23:37:02Z
2,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New entries translated to pt-BR,Igor Steinmacher,2013-10-24T19:22:22Z
3,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Adds missing beta 2 separator to the CHANGELOG,Oliver Kopp,2013-10-27T13:14:44Z
4,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Fixes string comparison at SQLUtil,Oliver Kopp,2013-10-27T13:17:00Z
5,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Update Japanese property files,Koji Yokota,2013-10-31T23:33:09Z
6,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Merge branch 'beta_2.10',Oliver Kopp,2013-11-04T08:35:17Z
7,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Merge branch 'master' of ssh://git.code.sf.net/p/jabref/code,Oliver Kopp,2014-02-12T12:09:43Z
8,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Fix GoogleScholarFetcher\n\nthe format of Google has changed and required an updated r...,Andreas Amann,2014-02-11T19:33:38Z
9,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Release of 2.10 beta 3,Oliver Kopp,2014-02-12T12:38:42Z


In [12]:
#result = pd.merge(user_usage, user_device[['use_id', 'platform', 'device']], on='use_id', how='outer',indicator=True)
result = pd.merge(dataIssues,
                 dataPullCommit,
                 on='PR_Number',
                 how='outer', 
                 indicator=True)

In [13]:
print(result.shape)
print(result.columns)
result.head(500)

(52968, 15)
Index(['PR_Number', 'Issue_Title', 'Issue_Author', 'Issue_Closed_Date',
       'Issue_Body', 'Issue_Comments', 'PR_Title', 'PR_Author',
       'PR_Closed_Date', 'PR_Body', 'PR_Comments', 'Commit_Message',
       'Commit_Author', 'Commit_Date', '_merge'],
      dtype='object')


,PR_Number,Issue_Title,Issue_Author,Issue_Closed_Date,Issue_Body,Issue_Comments,PR_Title,PR_Author,PR_Closed_Date,PR_Body,PR_Comments,Commit_Message,Commit_Author,Commit_Date,_merge
0,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,"Adding private fields to SQL scripts in order to export/import owner, timestamp, etc.",Igor Steinmacher,2013-10-23T19:13:12Z,both
1,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New entries translated to pt-BR,Igor Steinmacher,2013-10-23T23:37:02Z,both
2,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New entries translated to pt-BR,Igor Steinmacher,2013-10-24T19:22:22Z,both
3,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Adds missing beta 2 separator to the CHANGELOG,Oliver Kopp,2013-10-27T13:14:44Z,both
4,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Fixes string comparison at SQLUtil,Oliver Kopp,2013-10-27T13:17:00Z,both
5,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Update Japanese property files,Koji Yokota,2013-10-31T23:33:09Z,both
6,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Merge branch 'beta_2.10',Oliver Kopp,2013-11-04T08:35:17Z,both
7,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Merge branch 'master' of ssh://git.code.sf.net/p/jabref/code,Oliver Kopp,2014-02-12T12:09:43Z,both
8,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Fix GoogleScholarFetcher\n\nthe format of Google has changed and required an updated r...,Andreas Amann,2014-02-11T19:33:38Z,both
9,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Ex

In [14]:
result.rename(columns={'_merge':'isPR'}, inplace=True) #_merge is a column created by pd.merge()
result['isPR'].value_counts()


isPR
both          48872
left_only      4096
right_only        0
Name: count, dtype: int64

In [15]:
result['isPR'] = result['isPR'].apply({'both': 1, 'left_only': 0, 'right_only':0}.get)



In [16]:
result.to_csv('merged_Data.csv', sep=',', encoding='utf-8', index=False)


In [17]:
data = './merged_Data.csv'
data = pd.read_csv(data, sep=',' )


In [18]:
data.loc[0:100,'PR_Comments']
data.loc[98,]

data.loc[data['PR_Number'] == 5000]
print(data.shape)
data.head()

(52968, 15)


,PR_Number,Issue_Title,Issue_Author,Issue_Closed_Date,Issue_Body,Issue_Comments,PR_Title,PR_Author,PR_Closed_Date,PR_Body,PR_Comments,Commit_Message,Commit_Author,Commit_Date,isPR
0,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,"Adding private fields to SQL scripts in order to export/import owner, timestamp, etc.",Igor Steinmacher,2013-10-23T19:13:12Z,1
1,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New entries translated to pt-BR,Igor Steinmacher,2013-10-23T23:37:02Z,1
2,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New entries translated to pt-BR,Igor Steinmacher,2013-10-24T19:22:22Z,1
3,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Adds missing beta 2 separator to the CHANGELOG,Oliver Kopp,2013-10-27T13:14:44Z,1
4,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Fixes string comparison at SQLUtil,Oliver Kopp,2013-10-27T13:17:00Z,1


In [19]:
sum(pd.isnull(data['Issue_Closed_Date']))
data_final = data[data['Issue_Closed_Date'].notna()]

In [20]:
data_final.to_csv('dataMerged_Final_only_closed.csv', sep=',', encoding='utf-8', index=False)

In [21]:
data = './dataMerged_Final_only_closed.csv'
data = pd.read_csv(data, sep=',' )

In [23]:
data.head(100)

,PR_Number,Issue_Title,Issue_Author,Issue_Closed_Date,Issue_Body,Issue_Comments,PR_Title,PR_Author,PR_Closed_Date,PR_Body,PR_Comments,Commit_Message,Commit_Author,Commit_Date,isPR
0,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,"Adding private fields to SQL scripts in order to export/import owner, timestamp, etc.",Igor Steinmacher,2013-10-23T19:13:12Z,1
1,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New entries translated to pt-BR,Igor Steinmacher,2013-10-23T23:37:02Z,1
2,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New entries translated to pt-BR,Igor Steinmacher,2013-10-24T19:22:22Z,1
3,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Adds missing beta 2 separator to the CHANGELOG,Oliver Kopp,2013-10-27T13:14:44Z,1
4,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Fixes string comparison at SQLUtil,Oliver Kopp,2013-10-27T13:17:00Z,1
5,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Update Japanese property files,Koji Yokota,2013-10-31T23:33:09Z,1
6,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Merge branch 'beta_2.10',Oliver Kopp,2013-11-04T08:35:17Z,1
7,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Merge branch 'master' of ssh://git.code.sf.net/p/jabref/code,Oliver Kopp,2014-02-12T12:09:43Z,1
8,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,Fix GoogleScholarFetcher\n\nthe format of Google has changed and required an updated r...,Andreas Amann,2014-02-11T19:33:38Z,1
9,1,New Sorting/Export preferences,olenz,2014-03-12T09:26:29Z,"This will add a new ""File Sorting"" Tab to the preferences that allows to exactly speci...",NaN,New Sorting/Export preferences,olenz,2014-0

In [46]:
teste = './df_merge.csv'
testeMerge = pd.read_csv(teste,sep=',')
testeMerge

FileNotFoundError: [Errno 2] No such file or directory: './df_merge.csv'

In [12]:
#testeMerge.loc[7,]

#testeMerge.loc[testeMerge['PR_Number'] == 4498]

#testeMerge.loc[testeMerge['PR_Number'] == 4188]

testeMerge.loc[testeMerge['PR_Number'] == 452]

,PR_Number,Issue_Title,Issue_Body,Issue_Comments,PR_Comments,Commit_Message,isPR,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain
452,452,Revise maintable,Complete check and rewrite of MainTable and related classesThis will fix and implement...,=||= Awesome. =||= Before I invest much time to fix it...@JabRef/developers **Are we ...,=||= You could use Collections.singletonList instead. =||= Collections.emptyList =||...,minor changes,1,452.0,315.0,Rating colum has no header description,We should add a description here especially since clicking inside the column always op...,=||= The reason for directly changing the flags convenience for quick changing of fla...,0.0
453,452,Revise maintable,Complete check and rewrite of MainTable and related classesThis will fix and implement...,=||= Awesome. =||= Before I invest much time to fix it...@JabRef/developers **Are we ...,=||= You could use Collections.singletonList instead. =||= Collections.emptyList =||...,minor changes,1,452.0,384.0,Tooltip for title or long field,Adding a tooltip so we know the meaning of each column headtitle (R R Q F P is not obv...,=||= Thanks for your input. This issue is actively discussed among the developers so ...,0.0
454,452,Revise maintable,Complete check and rewrite of MainTable and related classesThis will fix and implement...,=||= Awesome. =||= Before I invest much time to fix it...@JabRef/developers **Are we ...,=||= You could use Collections.singletonList instead. =||= Collections.emptyList =||...,minor changes,1,452.0,443.0,GUI: Clicking on Ranking Col Header switches position of DOI column before/after Entry...,In 3.0 I just discovered a strange behavior:When I click on the Ranking Column Header ...,=||= Confirmed for 3.0 and `master`. =||= Fixed with #452 - changes are available now...,0.0
455,452,Revise maintable,Complete check and rewrite of MainTable and related classesThis will fix and implement...,=||= Awesome. =||= Before I invest much time to fix it...@JabRef/developers **Are we ...,=||= You could use Collections.singletonList instead. =||= Collections.emptyList =||...,minor changes,1,452.0,445.0,Moving ranking column: moved visually but function still in old place,In JabRef 3.0 I moved the ranking column to the end of my entry table using drag and d...,=||= Possibly #443 is related. =||= Fixed with #452 - changes are available now in cu...,0.0
456,452,Revise maintable,Complete check and rewrite of MainTable and related classesThis will fix and implement...,=||= Awesome. =||= Before I invest much time to fix it...@JabRef/developers **Are we ...,=||= You could use Collections.singletonList instead. =||= Collections.emptyList =||...,minor changes,1,452.0,498.0,Open pdf file when clicking pdf icon,NaN,=||= ... is working fine.Seriously you expect us to invest our time for free on somet...,0.0
457,452,Revise maintable,Complete check and rewrite of MainTable and related classesThis will fix and implement...,=||= Awesome. =||= Before I invest much time to fix it...@JabRef/developers **Are we ...,=||= You could use Collections.singletonList instead. =||= Collections.emptyList =||...,minor changes,1,452.0,490.0,Main Table: Right click on DOI field: Allow copying DOI,Currently the popup of DOI allows for clicking the DOI only. It should also be possibl...,=||= On which field? In field editor? See #492 =||= In the main table. At the DOI col...,0.0


In [11]:
testeMerge.loc[testeMerge['PR_Number'] == 677]

,PR_Number,Issue_Title,Issue_Body,Issue_Comments,PR_Comments,Commit_Message,isPR,PR,Issue,Issue_Title_Linked,Issue_Body_Linked,Issue_Comment_Linked,isTrain
681,677,Make BibTex parser more robust against missing newlines,As described in #669 and #621 the parser has some problems when an entry (or preamble)...,=||= :+1: (except my minor comment) =||= Corrected... for some reason the code style ...,=||= Please always use curly braces as this makes the semantics explicit.,Minor code reformatting,1,677.0,669.0,JabRef 3.1/3.2 writes bib file in a format that it will not read,If the @PREAMBLE field is present JabRef 3.1/3.2 omits the newline between that line a...,=||= related #621 =||= @tobiasdiez Is this fixed? =||= @stefan-kolb It will be fixed ...,0.0
682,677,Make BibTex parser more robust against missing newlines,As described in #669 and #621 the parser has some problems when an entry (or preamble)...,=||= :+1: (except my minor comment) =||= Corrected... for some reason the code style ...,=||= Please always use curly braces as this makes the semantics explicit.,Minor code reformatting,1,677.0,621.0,Ver 3.1 comment not written correctly.,I have a file started with bibtex item e.g.1 @ARTICLE{XXXXXX2 author = {XXXXXXXX...,=||= This is not a bug but the exact desired behavior.We only modify entries that hav...,0.0
683,677,Make BibTex parser more robust against missing newlines,As described in #669 and #621 the parser has some problems when an entry (or preamble)...,=||= :+1: (except my minor comment) =||= Corrected... for some reason the code style ...,=||= Please always use curly braces as this makes the semantics explicit.,Minor code reformatting,1,677.0,669.0,JabRef 3.1/3.2 writes bib file in a format that it will not read,If the @PREAMBLE field is present JabRef 3.1/3.2 omits the newline between that line a...,=||= related #621 =||= @tobiasdiez Is this fixed? =||= @stefan-kolb It will be fixed ...,0.0
684,677,Make BibTex parser more robust against missing newlines,As described in #669 and #621 the parser has some problems when an entry (or preamble)...,=||= :+1: (except my minor comment) =||= Corrected... for some reason the code style ...,=||= Please always use curly braces as this makes the semantics explicit.,Minor code reformatting,1,677.0,621.0,Ver 3.1 comment not written correctly.,I have a file started with bibtex item e.g.1 @ARTICLE{XXXXXX2 author = {XXXXXXXX...,=||= This is not a bug but the exact desired behavior.We only modify entries that hav...,0.0
